In [2]:
# Import required libraries
import pandas as pd
import numpy as np

def preprocess_dataset(filepath):
    df = pd.read_csv(filepath)

    # Remove duplicates
    df = df.drop_duplicates()

    # Strip whitespace from string columns
    for col in df.select_dtypes(include=['object']).columns:
        df[col] = df[col].str.strip()

    # Handle missing values
    for col in df.columns:
        if df[col].dtype in [np.float64, np.int64]:
            df[col] = df[col].fillna(df[col].median())
        else:
            df[col] = df[col].fillna(df[col].mode()[0] if not df[col].mode().empty else '')

    return df


def update_campaign_channel(filepath):    
    # Expand 'CampaignChannel' values if column exists
    df = pd.read_csv(filepath)
    if 'CampaignChannel' in df.columns:
        social_media_platforms = ['whatsapp', 'instagram', 'facebook', 'twitter', 'linkedin', 'tiktok']
        email_providers = ['outlook', 'gmail', 'yahoo', 'zoho', 'protonmail']
        ppc_platforms = ['google ads', 'bing ads', 'facebook ads', 'linkedin ads', 'twitter ads']
        seo_engines = ['google', 'bing', 'yahoo', 'duckduckgo', 'baidu']
        referral_sources = ['partner site', 'blog', 'news portal', 'affiliate', 'online directory']
        df['CampaignChannel'] = df['CampaignChannel'].replace(['social media', 'Social Media', 'SOCIAL MEDIA'], 'social media')
        df['CampaignChannel'] = df['CampaignChannel'].replace(['email', 'Email', 'EMAIL'], 'email')
        df['CampaignChannel'] = df['CampaignChannel'].replace(['ppc', 'PPC', 'Pcc', 'PCC'], 'ppc')
        df['CampaignChannel'] = df['CampaignChannel'].replace(['seo', 'SEO', 'Seo'], 'seo')
        df['CampaignChannel'] = df['CampaignChannel'].replace(['referral', 'Referral', 'REFERRAL'], 'referral')
        sm_mask = df['CampaignChannel'] == 'social media'
        em_mask = df['CampaignChannel'] == 'email'
        ppc_mask = df['CampaignChannel'] == 'ppc'
        seo_mask = df['CampaignChannel'] == 'seo'
        referral_mask = df['CampaignChannel'] == 'referral'
        df.loc[sm_mask, 'CampaignChannel'] = np.random.choice(social_media_platforms, size=sm_mask.sum())
        df.loc[em_mask, 'CampaignChannel'] = np.random.choice(email_providers, size=em_mask.sum())
        df.loc[ppc_mask, 'CampaignChannel'] = np.random.choice(ppc_platforms, size=ppc_mask.sum())
        df.loc[seo_mask, 'CampaignChannel'] = np.random.choice(seo_engines, size=seo_mask.sum())
        df.loc[referral_mask, 'CampaignChannel'] = np.random.choice(referral_sources, size=referral_mask.sum())
    df.to_csv('digital_marketing_campaign_dataset.csv')
    

# Example usage:
df_clean = preprocess_dataset('digital_marketing_campaign_dataset.csv')
df_clean.to_csv('filtered.csv', index=False)
update_campaign_channel('digital_marketing_campaign_dataset.csv')

In [3]:
def reverse_campaign_channel(filepath, column_name=None):
    """
    Reverse the effect of update_campaign_channel for the specified column, or auto-detect the column if not provided.
    Maps platforms/sources back to their generic category.
    """
    df = pd.read_csv(filepath)
    mapping = {
        # Social media
        'whatsapp': 'social media', 'instagram': 'social media', 'facebook': 'social media',
        'twitter': 'social media', 'linkedin': 'social media', 'tiktok': 'social media',
        # Email
        'outlook': 'email', 'gmail': 'email', 'yahoo': 'email', 'zoho': 'email', 'protonmail': 'email',
        # PPC
        'google ads': 'ppc', 'bing ads': 'ppc', 'facebook ads': 'ppc', 'linkedin ads': 'ppc', 'twitter ads': 'ppc',
        # SEO
        'google': 'seo', 'bing': 'seo', 'yahoo': 'seo', 'duckduckgo': 'seo', 'baidu': 'seo',
        # Referral
        'partner site': 'referral', 'blog': 'referral', 'news portal': 'referral', 'affiliate': 'referral', 'online directory': 'referral'
    }
    # If column_name is not provided, auto-detect the column containing any mapping value
    if column_name is None:
        mapping_values = set(mapping.keys())
        for col in df.columns:
            if df[col].dtype == object:
                if df[col].str.lower().isin(mapping_values).any():
                    column_name = col
                    break
    if column_name and column_name in df.columns:
        df[column_name] = df[column_name].map(lambda x: mapping.get(str(x).strip().lower(), x))
    df.to_csv('reversed.csv', index=False)
    return df

In [19]:
reverse_campaign_channel('digital_marketing_campaign_dataset.csv')

,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,CustomerID,Age,Gender,Income,CampaignChannel,CampaignType,...,PagesPerVisit,TimeOnSite,SocialShares,EmailOpens,EmailClicks,PreviousPurchases,LoyaltyPoints,AdvertisingPlatform,AdvertisingTool,Conversion
0,0,0,0,0,8000,56,Female,136912,social media,Awareness,...,2.399017,7.396803,19,6,9,4,688,BuySellAds,Amplitude,1
1,1,1,1,1,8001,69,Male,41760,email,Retention,...,2.917138,5.352549,5,2,7,2,3459,Instagram Ads,Drip,1
2,2,2,2,2,8002,46,Female,88456,ppc,Awareness,...,8.223619,13.794901,0,11,2,8,2337,Yahoo Gemini,LeadSquared,1
3,3,3,3,3,8003,32,Female,44085,ppc,Conversion,...,4.540939,14.688363,89,2,2,0,2463,Bing Ads,Heap Analytics,1
4,4,4,4,4,8004,60,Female,83964,ppc,Conversion,...,2.046847,13.993370,6,6,6,8,4345,MoPub,Hootsuite,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,7995,7995,7995,7995,15995,21,Male,24849,email,Awareness,...,9.693602,14.227794,70,13,6,7,286,Fyber,SEMrush,0
7996,7996,7996,7996,7996,15996,43,Female,44718,seo,Retention,...,9.499010,3.501106,52,13,1,5,1502,Amazon Ads,AdEspresso,0
7997,7997,7997,7997,7997,15997,28,Female,125471,referral,Consideration,...,2.853241,14.618323,38,16,0,3,738,Apple Search Ads,LeadSquared,1
7998,7998,7998,7998,7998,15998,19,Female,107862,ppc,Consideration,...,1.002964,3.876623,86,1,5,7,2709,AdColony,Canva Pro,1


In [4]:
import pandas as pd
import numpy as np
import math
import re
import hashlib

# Generalization function
def generalize_integer_column_auto(series):
    if not pd.api.types.is_integer_dtype(series):
        raise ValueError("Input series must be integer type")
    
    n = len(series)
    if n == 0:
        return series  # empty series, return as is
    
    data_min = series.min()
    data_max = series.max()
    data_range = data_max - data_min
    
    if data_range == 0:
        return pd.Series([f"{data_min}-{data_min}"] * n, index=series.index)
    
    num_bins = math.ceil(1 + math.log2(n))  # Sturges' formula
    bucket_size = math.ceil(data_range / num_bins)
    
    def to_bucket(val):
        start = ((val - data_min) // bucket_size) * bucket_size + data_min
        end = start + bucket_size - 1
        return f"{start}-{end}"
    
    return series.apply(to_bucket)

# Type inference


def guess_column_type(series: pd.Series) -> str:
    col_name = series.name.lower()
    values = series.dropna()

    # Prioritize checking dtype first
    if pd.api.types.is_integer_dtype(values):
        # Even if unique ratio is low, treat as integer (if desired)
        return 'integer'
    
    if pd.api.types.is_float_dtype(values):
        return 'float'
    
    # Then check column name hints (optional, for more accuracy)
    if any(keyword in col_name for keyword in ['date', 'time', 'year', 'month', 'day']):
        return 'datetime'
    if any(keyword in col_name for keyword in ['flag', 'is_', 'has_', 'bool', 'binary']):
        return 'boolean'
    
    # Then fallback to unique ratio heuristic for categoricals
    if pd.api.types.is_object_dtype(values):
        unique_ratio = values.nunique() / len(values)
        if unique_ratio < 0.1:
            return 'categorical'
        else:
            return 'text'
    
    # Default fallback
    return 'unknown'

def guess_types_from_dataframe(df: pd.DataFrame) -> dict:
    inferred_types = {}
    for col in df.columns:
        inferred_types[col] = guess_column_type(df[col])
    return inferred_types
    

# Sensitivity classification
def classify_attribute(series: pd.Series) -> str:
    col_name = series.name.lower()
    values = series.dropna()

    # Treat any column with 'id' in the name as Quasi-Identifier
    if 'id' in col_name:
        return 'Quasi-Identifier'

    # Updated keyword categories
    sensitive_keywords = [
        'ssn', 'socialsecurity', 'creditcard', 'password', 'phone',
        'health', 'medical', 'salary', 'income', 'platform', 'tool'
    ]
    quasi_identifier_keywords = [
        'age', 'dob', 'dateofbirth', 'zipcode', 'postalcode', 'city',
        'state', 'gender', 'race', 'ethnicity'
    ]
    known_normal_keywords = [
        'visit', 'click', 'share', 'point', 'spend', 'rate',
        'conversion', 'purchase', 'loyalty'
    ]

    # Rule-based keyword checks
    if any(keyword in col_name for keyword in sensitive_keywords):
        return 'Sensitive'
    if any(keyword in col_name for keyword in known_normal_keywords):
        return 'Normal'
    if any(keyword in col_name for keyword in quasi_identifier_keywords):
        return 'Quasi-Identifier'

    # Numeric type analysis
    if pd.api.types.is_numeric_dtype(values):
        unique_ratio = values.nunique() / len(values)
        if unique_ratio > 0.1:
            return 'Normal'
        else:
            return 'Quasi-Identifier'

    # Object/text type analysis
    if pd.api.types.is_object_dtype(values):
        unique_ratio = values.nunique() / len(values)
        if unique_ratio < 0.1:
            return 'Quasi-Identifier'
        else:
            return 'Normal'

    # Fallback
    return 'Normal'

def mask_sensitive_column(series: pd.Series) -> pd.Series:
    return pd.Series(["isConfid"] * len(series), index=series.index)

def pseudonymize_customer_ids(data):
    df = data
    unique_ids = df.unique()
    pseudonym_map = {real_id: f"CUST{idx:04d}" for idx, real_id in enumerate(unique_ids, start=1)}
    df = df.map(pseudonym_map)
    return df
def encode_gender_column(series: pd.Series) -> pd.Series:
    return series.str.strip().str.lower().map({
        'male': 1,
        'female': 0
    })

def generate_seed(value, secret_key="my_secret"):
    hash_input = f"{value}_{secret_key}".encode()
    hashed = hashlib.sha256(hash_input).hexdigest()
    return int(hashed, 16) % (2**32)

def deterministic_noise(value, std_dev, noise_level=0.05, secret_key="my_secret"):
    seed = generate_seed(value, secret_key)
    rng = np.random.default_rng(seed)
    noise = rng.normal(loc=0, scale=noise_level * std_dev)
    return noise

def reversible_perturb(series: pd.Series, noise_level=0.05, secret_key="my_secret") -> pd.Series:
    std_dev = series.std()
    perturbed = []

    for val in series:
        if pd.isna(val):
            perturbed.append(np.nan)
            continue
        noise = deterministic_noise(val, std_dev, noise_level, secret_key)
        perturbed.append(val + noise)

    return pd.Series(perturbed, index=series.index)
    
def reverse_perturb(perturbed_series: pd.Series, original_series: pd.Series, noise_level=0.05, secret_key="my_secret") -> pd.Series:
    std_dev = original_series.std()
    reversed_vals = []

    for perturbed_val, original_val in zip(perturbed_series, original_series):
        if pd.isna(perturbed_val) or pd.isna(original_val):
            reversed_vals.append(np.nan)
            continue
        noise = deterministic_noise(original_val, std_dev, noise_level, secret_key)
        reversed_val = perturbed_val - noise
        reversed_vals.append(reversed_val)

    return pd.Series(reversed_vals, index=perturbed_series.index)

def generalize_integer_column_fixed_bin(series: pd.Series, bin_size=5) -> pd.Series:
    if not pd.api.types.is_integer_dtype(series):
        raise ValueError("Input series must be integer type")
    
    n = len(series)
    if n == 0:
        return series  # empty series, return as is
    
    data_min = series.min()
    data_max = series.max()
    data_range = data_max - data_min
    
    # Handle edge case: all values same
    if data_range == 0:
        return pd.Series([f"{data_min}-{data_min}"] * n, index=series.index)
    
    def to_bucket(val):
        start = ((val - data_min) // bin_size) * bin_size + data_min
        end = start + bin_size - 1
        return f"{start}-{end}"
    
    generalized = series.apply(to_bucket)
    return generalized

df = pd.read_csv("digital_marketing_campaign_dataset.csv")
types = {col: guess_column_type(df[col]) for col in df.columns}
classification = {col: classify_attribute(df[col]) for col in df.columns}
# print(df['PagesPerVisit'])

for col in df.columns:
    if classification[col] in ['Quasi-Identifier', 'Sensitive'] and types[col] == 'integer' and col!="CustomerID":
        df[col] = generalize_integer_column_auto(df[col])
    elif classification[col] in ['Quasi-Identifier', 'Sensitive'] and types[col] == 'integer' and col=="CustomerID":
        df[col]=pseudonymize_customer_ids(df[col])
    elif classification[col] in ['Quasi-Identifier', 'Sensitive'] and types[col] == 'categorical' and col in ["AdvertisingPlatform","AdvertisingTool"]:
        df[col]=mask_sensitive_column(df[col])
    elif col=='Gender':
        df[col]=encode_gender_column(df[col])
    elif types[col] == 'float':
        df[col]=reversible_perturb(df[col])
    elif col=='PreviousPurchases':
        df[col] = generalize_integer_column_fixed_bin(df[col])

# Save to a new file if needed
df.to_csv("answer.csv", index=False)
# print(df)


In [6]:
import pandas as pd
def check_l_diversity(df, quasi_identifiers, sensitive_col, l=2):
    """
    Check l-diversity for each group defined by quasi-identifiers.
    Returns a DataFrame with group sizes and number of unique sensitive values, and a boolean column for l-diversity.
    """
    grouped = df.groupby(quasi_identifiers)[sensitive_col].agg(['count', pd.Series.nunique]).reset_index()
    grouped = grouped.rename(columns={'nunique': 'unique_sensitive'})
    grouped['l_diverse'] = grouped['unique_sensitive'] >= l
    return grouped

# Example usage:
df = pd.read_csv('answer_sorted.csv')
result = check_l_diversity(df, ['Age', 'Income'], 'EmailOpens', l=2)  # Replace columns as needed
print(result)

       Age         Income  count  unique_sensitive  l_diverse
0    18-21  103570-112853     48                10       True
1    18-21  112854-122137     49                10       True
2    18-21  122138-131421     40                 9       True
3    18-21  131422-140705     54                 9       True
4    18-21  140706-149989     35                 9       True
..     ...            ...    ...               ...        ...
177  66-69    57150-66433     32                10       True
178  66-69    66434-75717     39                10       True
179  66-69    75718-85001     53                10       True
180  66-69    85002-94285     42                10       True
181  66-69   94286-103569     39                10       True

[182 rows x 5 columns]


In [7]:
import pandas as pd

def enforce_l_diversity(df, quasi_identifiers, sensitive_col, l=2):
    df = df.copy()
    grouped = df.groupby(quasi_identifiers)
    for group_keys, group_indices in grouped.groups.items():
        group = df.loc[group_indices]
        unique_vals = group[sensitive_col].nunique()
        if unique_vals < l:
            # Get the base value(s) to modify
            base_vals = group[sensitive_col].unique()
            for i, idx in enumerate(group_indices):
                # Cycle through base values and append a suffix to make them unique
                new_val = f"{base_vals[i % len(base_vals)]}_var{i+1}"
                df.at[idx, sensitive_col] = new_val
    return df

# Example usage:
df = pd.read_csv('answer_sorted.csv')
result = check_l_diversity(df, ['Age', 'Income'], 'EmailOpens', l=2)
df_diverse = enforce_l_diversity(df, ['Age', 'Income'], 'EmailOpens', l=2)
result_after = check_l_diversity(df_diverse, ['Age', 'Income'], 'EmailOpens', l=2)